#### Deploy ONNX model as a WebService in the Azure Cloud<br>
<br>
There are four steps which are needed to follow when you deploy your model.<br>
1. Register the model.<br>
2. Prepare to deploy. (Specify assets, usage, compute target).<br>
3. Deploy the model to the compute target.<br>
4. Test the deployed model, also called a web service.<br>

###### Initialize Azure Workspace

In [4]:
from azureml.core import Workspace
ws = Workspace.from_config(r'C:\Users\ASUS\Desktop\app')

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


###### Register your ONNX model with Azure ML

In [6]:
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "C:/Users/ASUS/Desktop/app/models",
                       model_name = "house.onnx",
                       description = "Boston house model trained outside Azure Machine Learning",
                       workspace = ws)

Registering model house.onnx


#### Prepare to Deploy
###### 1. Define inference configuration Environment file<br>
Defining dependencies that are required to run the model

In [20]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_pip_package("pydantic")
conda_dep.add_pip_package("numpy")
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("skl2onnx")
conda_dep.add_pip_package("onnxruntime")
conda_dep.add_pip_package("sklearn")

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep



###### 2. Define scoring code<br>
An entry script. This file (named score.py) loads the model when the deployed service starts. It is also responsible for receiving data, passing it to the model, and then returning a response.

In [40]:
inference_config = InferenceConfig(entry_script="C:/Users/ASUS/Desktop/app/score.py",environment=myenv)

###### 3. Define inference configuration<br>
Defining the deployment target class.Here i used Azure Container Instance servive class as webservice.Used for low-scale CPU-based workloads that require less than 48 GB of RAM.

In [41]:
from azureml.core.webservice import AciWebservice, Webservice


deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)


In [42]:
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print("scoring URI: ",service.scoring_uri)

Running..............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI:  http://5f00c0fd-1012-4d32-b6da-dbeb4b2c5d39.centralus.azurecontainer.io/score


In [43]:
# service.get_logs()

###### Access the response

In [47]:
import requests
import json

headers = {'Content-Type': 'application/json'}
response = requests.post(
    service.scoring_uri, json = [0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67], headers=headers)

In [49]:
response.json()

{'prediction': 23.21451187133789}

##### Conclusion<br>
Therefore, we deployed our model as a web service and consumed the response back as a price provided the request with the features of the house.

###### References
<br>https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model<br>
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where<br>
https://towardsdatascience.com/deploy-your-machine-learning-model-as-a-rest-api-4fe96bf8ddcc